# 用RNN自动生成中文现代诗词

## Helper functions

In [37]:
#coding=utf-8
import pickle
import os
import re
def load_text(path):
    input_file = os.path.join(path)

    with open(input_file, 'r') as f:
        text_data = f.read()

    return text_data

def preprocess_and_save_data(text, token_lookup, create_lookup_tables):
    token_dict = token_lookup()
    
    for key, token in token_dict.items():
        text = text.replace(key, '{}'.format(token))
    text = list(text)
    
    vocab_to_int, int_to_vocab = create_lookup_tables(text)
    int_text = [vocab_to_int[word] for word in text]
    
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))

def load_preprocess():
    return pickle.load(open('preprocess.p', mode='rb'))

def save_params(params):
    pickle.dump(params, open('params.p', 'wb'))

def load_params():
    return pickle.load(open('params.p', mode='rb'))

In [338]:
data_dir = '~/Documents/fastai-notes/deeplearning1/rnn/data/'
filename = 'gucheng.txt'
%cd $data_dir
text = load_text(filename)

/Users/PeiYingchi/Documents/fastai-notes/deeplearning1/rnn/data


In [339]:
text[:500]

'\n\n松\u3000塔\n松枝上，\n露滴晶光闪亮，\n好像绿漆的宝塔，\n挂满银铃铛。\n\n杨\u3000树\n我失去了一只臂膀，\n就睁开了一只眼睛。\n \n\n黄 昏\n猛烈的北风，\n吹散了人们淡薄的脚印；\n太阳落山了，\n世界像是一幅巨大的剪影。\n\n烟 囱\n烟囱犹如平地耸立起来的巨人，\n望着布满灯火的大地，\n不断地吸着烟卷，\n思索着一种谁也不知道的事情。\n\n\n星 月 的 来 由\n树枝想去撕裂天空，\n但却只戳了几个微小的窟窿，\n它透出了天外的光亮，\n人们把它叫作月亮和星星。\n\n塔 和 晨\n洁白的塔呵，\n围着绿色的腰带，\n像一枝春天的竹笋，\n在召唤满天蓬松的云彩。\n这是一个美丽的晨景，\n到处都悬着露水，\n像无数儿童的眼睛。\n在湿湿的霞光里，\n水光映着铜铃，\n铃响伴着和风。\n在云雾消散的松林里，\n回荡着啄木鸟工作的歌声。\n1968年\n天\n白云是天的雪山；\n碧空是天的海洋；\n阳光是天的熔岩；\n阴霾是天的煤矿；\n星团是天的城市；\n流星是天的车辆；\n天上的一切只能遥遥相望，\n所以天是幻想的家乡。\n \n上一页目\u3000录下一页\n\n□ 版权所有――顾城\n\n\n\n\n\n  \n\n\n \n\n\n\n\n  \n\n一九六九年\n \n我 的 幻 想\n我在幻想着，\n幻想'

In [340]:
text = text.replace(' ', '').replace('……', '').replace('\u3000','')

In [341]:
text = re.sub('(19.*?)\\n','\n', text)
text = re.sub('(一九.*?)\\n','\n', text)

In [342]:
text = re.sub(r'\n{2,}', '&', text)

In [343]:
text[:500]

'&松塔\n松枝上，\n露滴晶光闪亮，\n好像绿漆的宝塔，\n挂满银铃铛。&杨树\n我失去了一只臂膀，\n就睁开了一只眼睛。&黄昏\n猛烈的北风，\n吹散了人们淡薄的脚印；\n太阳落山了，\n世界像是一幅巨大的剪影。&烟囱\n烟囱犹如平地耸立起来的巨人，\n望着布满灯火的大地，\n不断地吸着烟卷，\n思索着一种谁也不知道的事情。&星月的来由\n树枝想去撕裂天空，\n但却只戳了几个微小的窟窿，\n它透出了天外的光亮，\n人们把它叫作月亮和星星。&塔和晨\n洁白的塔呵，\n围着绿色的腰带，\n像一枝春天的竹笋，\n在召唤满天蓬松的云彩。\n这是一个美丽的晨景，\n到处都悬着露水，\n像无数儿童的眼睛。\n在湿湿的霞光里，\n水光映着铜铃，\n铃响伴着和风。\n在云雾消散的松林里，\n回荡着啄木鸟工作的歌声。&天\n白云是天的雪山；\n碧空是天的海洋；\n阳光是天的熔岩；\n阴霾是天的煤矿；\n星团是天的城市；\n流星是天的车辆；\n天上的一切只能遥遥相望，\n所以天是幻想的家乡。&上一页目录下一页&□版权所有――顾城&我的幻想\n我在幻想着，\n幻想在破灭着；\n幻想总把破灭宽恕，\n破灭却从不把幻想放过。&美\n我所渴望的美，\n是永恒与生命；\n谁知它们竟水火不容。\n永恒的美'

In [344]:
text = re.sub('(&[\u4300-\u9fff]+)\n([\u4300-\u9fff])', r'\1:\2', text)

In [345]:
text[:500]

'&松塔:松枝上，\n露滴晶光闪亮，\n好像绿漆的宝塔，\n挂满银铃铛。&杨树:我失去了一只臂膀，\n就睁开了一只眼睛。&黄昏:猛烈的北风，\n吹散了人们淡薄的脚印；\n太阳落山了，\n世界像是一幅巨大的剪影。&烟囱:烟囱犹如平地耸立起来的巨人，\n望着布满灯火的大地，\n不断地吸着烟卷，\n思索着一种谁也不知道的事情。&星月的来由:树枝想去撕裂天空，\n但却只戳了几个微小的窟窿，\n它透出了天外的光亮，\n人们把它叫作月亮和星星。&塔和晨:洁白的塔呵，\n围着绿色的腰带，\n像一枝春天的竹笋，\n在召唤满天蓬松的云彩。\n这是一个美丽的晨景，\n到处都悬着露水，\n像无数儿童的眼睛。\n在湿湿的霞光里，\n水光映着铜铃，\n铃响伴着和风。\n在云雾消散的松林里，\n回荡着啄木鸟工作的歌声。&天:白云是天的雪山；\n碧空是天的海洋；\n阳光是天的熔岩；\n阴霾是天的煤矿；\n星团是天的城市；\n流星是天的车辆；\n天上的一切只能遥遥相望，\n所以天是幻想的家乡。&上一页目录下一页&□版权所有――顾城&我的幻想:我在幻想着，\n幻想在破灭着；\n幻想总把破灭宽恕，\n破灭却从不把幻想放过。&美:我所渴望的美，\n是永恒与生命；\n谁知它们竟水火不容。\n永恒的美'

In [346]:
text = text.replace('(一)',':')
text = text.replace('(二)',':')
text = text.replace('(三)',':')

In [347]:
text = text.replace('\n', ' ').replace(u'目\u3000录', '')

In [348]:
text[:500]

'&松塔:松枝上， 露滴晶光闪亮， 好像绿漆的宝塔， 挂满银铃铛。&杨树:我失去了一只臂膀， 就睁开了一只眼睛。&黄昏:猛烈的北风， 吹散了人们淡薄的脚印； 太阳落山了， 世界像是一幅巨大的剪影。&烟囱:烟囱犹如平地耸立起来的巨人， 望着布满灯火的大地， 不断地吸着烟卷， 思索着一种谁也不知道的事情。&星月的来由:树枝想去撕裂天空， 但却只戳了几个微小的窟窿， 它透出了天外的光亮， 人们把它叫作月亮和星星。&塔和晨:洁白的塔呵， 围着绿色的腰带， 像一枝春天的竹笋， 在召唤满天蓬松的云彩。 这是一个美丽的晨景， 到处都悬着露水， 像无数儿童的眼睛。 在湿湿的霞光里， 水光映着铜铃， 铃响伴着和风。 在云雾消散的松林里， 回荡着啄木鸟工作的歌声。&天:白云是天的雪山； 碧空是天的海洋； 阳光是天的熔岩； 阴霾是天的煤矿； 星团是天的城市； 流星是天的车辆； 天上的一切只能遥遥相望， 所以天是幻想的家乡。&上一页目录下一页&□版权所有――顾城&我的幻想:我在幻想着， 幻想在破灭着； 幻想总把破灭宽恕， 破灭却从不把幻想放过。&美:我所渴望的美， 是永恒与生命； 谁知它们竟水火不容。 永恒的美'

In [349]:
text = re.sub('(上一页.*?顾城)&', '', text)

In [350]:
text = text.replace('&', '\n')
text = re.sub('([\u4300-\u9fff]) ([\u4300-\u9fff])', r'\1，\2', text)
text = text.replace(' ', '')

In [355]:
print(text[25000:26000])

钱，不要那些不会发芽的分币，我只要装满小小的花籽，我要知道她们的生日
水呀，真急，水呀，真急，真急，桥墩后有几条小鱼，它们在举行会议，研究着前进还是退避。太阳在桥面上走过，带着几分醉意。研究在不断继续，河水在不断流去。月牙在桥栏边停靠，似乎要看个仔细。水呵，真急、真急，桥墩后有几条小鱼
比萨斜塔――此时，它仍在倾斜，祖先聪慧的民族，难道毫无办法？塔，倾斜的影子，投在我不平的心上，弯弯曲曲，游客在大声哗笑。崩溃在逼近――一分一秒、一丝一厘，也许还要一个世纪。但游客只需要一分钟。转机，转机在哪里？夕阳落下了，暮色把灾难的预言抹去。游客们该满足了吧！是的，我不满足，因为我活得太久，不愿看到一片废墟。金字塔――献给你的赞美诗，也能筑一座塔了吧？可惜我并不是诗人。是梦里的风，把我吹去――我落在你的脚下，仰看着闪电的足迹，仰看着岩缝中的云，我是一粒砂子。你伟大，你像盘集的长城，但为什么，仅仅一块方石，就把你全部镇压？你是一座坟墓。
我的独木船:一，我的独木船，没有桨，没有风帆，飘在大海中间，飘在大海中间，没有桨，没有风帆。风呵，命运的风呵，感情的波澜，请把我吞没，或送回彼岸，即使是梦幻，即使是梦幻，我在盼望那，沉静的港湾；我在盼望那，黄金的海滩；我在盼望那，岸边的姑娘，和她相见，和她相见，和她相见！二，我的独木船，没有舵，没有绳缆，飘在人世间，飘在人世间，没有舵，没有绳缆，风呵，命运的风呵，生活的波澜，请把我埋藏，或送回家园，即使是碎片，即使是碎片，我在想念那，美丽的栈桥；我在想念那，含泪的灯盏；我在想念那，灯下的母亲，祝她晚安，祝她晚安，祝她晚安。
我要去见她:小小的星儿，你瞧什么？我要去见她。含羞的云儿，你笑什么？我要去见她。细细的树儿，你等什么？我要去见她。悄悄的风儿，你跑什么？我要去见她。矮矮的篱笆，跳过去啦，我要去见她。浅浅的河水，过去啦，我要去见她。高高的山坡，登上去啦，我要去见她。宽宽的大海，游过去啦，我要去见她！
梦想:种子在冻土里，梦想着春天。它梦见――自己舒展着颤动的腰身，长睫旁闪耀着露滴的银钻；它梦见――蝴蝶轻轻地吻它，春蚕张开了新房的金幔；它梦见――无数花朵睁开了稚气的眼睛，就像月亮身边的万千星点，种子呵，在冻土里梦想春天
初春:阴沉的天空在犹豫：是雪花？还是雨滴？混浊的河流在疾走：是追求？还是逃避？远处的情侣在分别：是序幕？还是结局？
梦:春天里，花占

In [352]:
with open('gucheng_compiled.txt', 'w') as out_file:
    out_file.write(text)